<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Credit Card Fraud Detection - Data Cleansing and Feature Engineering Pipeline</b>
</header>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial'>
This is a demonstration of Vantage capabilities for functional demos e.g.
    <li style = 'font-size:16px;font-family:Arial'> Data Cleansing Functions - like  TD_GetFutileColumns, TD_SimpleImputeFit and TD_SimpleImputeTransform </li>
        <li style = 'font-size:16px;font-family:Arial'> Data Exploration Functions - like  TD_ColumnSummary and TD_CategoicalSummary </li>
    <li style = 'font-size:16px;font-family:Arial'> Feature Engineering Functions - like TD_BinCodeFit & Transform, TD_OrdinalEncodingFit & Transform, TD_OnehotencodingFit & Transform, TD_ScaleFit & Transform and TD_ColumnTransformer </li>
</p>
<br>
<p style = 'font-size:16px;font-family:Arial'>
In a typical Data Science project there are multiple pre-processing steps involved to process the raw incoming data before it can actually be used in a model for predictions. On an estimate about 70-80% of the time and effort goes into the pre-processing steps. With the help of Vantage's in Db functions we can perform these functions very effectively and at scale.
In this demo notebook we are using a sample financial data of credit card application with target of loan defaulters, we will go through the general pre-processing steps that are involved in getting the source data and making the data useable for model creation.
</p>  

<p style = 'font-size:28px;font-family:Arial;color:#E37C4D'><b>1. Connect to Vantage and explore the dataset</b></p>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b>Let's start by connecting to the Teradata system </b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press Enter, then use down arrow to go to next cell.</p>

In [1]:
%connect local, hidewarnings=true

Password: ········


Success: 'local' connection established and activated for user 'demo_user', with default database 'demo_user'


<p style = 'font-size:16px;font-family:Arial'>Setup for execution of notebook. Begin running steps with Shift + Enter keys.</p>

In [2]:
Set query_band='DEMO=CreditCardFraud.ipynb;' update for session;

Success: 1 rows affected

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b>Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. In this demo since we are using Temporal table we will be creating databases and tables in local storage and use them in the notebook. Please execute the procedure in the next cell.</p>


In [3]:
call get_data('DEMO_CreditCard_cloud');    -- takes about 20 seconds, estimated space: 0 MB
--call get_data('DEMO_CreditCard_local');     -- takes about 35 seconds, estimated space: 11 MB

Success: 0 rows affected

Success: 0 rows affected

,Message
1,That ran for 0:00:11.64 with 5 statements and 0 errors.


<p style = 'font-size:16px;font-family:Arial'>Optional step – if you want to see status of databases/tables created and space used.</p>

In [4]:
call space_report();  -- optional, takes about 10 seconds

Success: 0 rows affected

Success: 0 rows affected

,Space_Report
1,"You have: #databases=1 #tables=7 #views=8 You have used 1.7 MB of 27,914.4 MB available - 0.0% ... Space Usage OK"
2,
3,Database Name #tables #views Avail MB Used MB
4,"demo_user 7 7 27,914.4 MB 1.7 MB"
5,DEMO_CreditCard 0 1 0.0 MB 0.0 MB


<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b> Access data in Vantage  </b> </p>
<p style = 'font-size:16px;font-family:Arial'>For this demo, data is already resident in Object Storage which we are accessing via ReadNOS, create a reference to the table, and sample the contents using the get_data procedure used above.  Data could just as easily reside in permanent tables, another RDBMS, or another Vantage system.</p>

In [5]:
sel top 5 * from DEMO_CreditCard.Credit_Card;

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_FAMILY_STATUS,REGION_POPULATION_RELATIVE,FLAG_MOBIL,FLAG_EMP_PHONE,CNT_FAM_MEMBERS,HOUSETYPE_MODE,OCCUPATION_TYPE,AGE
1,357714,1,Cash loans,M,N,2,180000,Married,0.03579200059175491,1,0,4,block of flats,,37
2,119006,0,Cash loans,M,N,2,180000,Separated,0.01014699973165989,1,1,3,,,34
3,143600,1,Cash loans,F,N,0,90000,Civil marriage,0.02280000038444996,1,1,2,,Cleaning staff,27
4,126899,0,Cash loans,F,N,2,180000,Married,0.025164000689983368,1,1,4,block of flats,Sales staff,32
5,295785,1,Cash loans,F,N,1,99000,Married,0.009657000191509724,1,1,3,block of flats,Core staff,43


<p style = 'font-size:28px;font-family:Arial;color:#E37C4D'> <b> 2. Checking data demographics  </b> </p>
<p style = 'font-size:16px;font-family:Arial'><b>TD_ColumnSummary </b>function  displays Column name, datatype and other demographics like count of NULLs etc for each specified input table column</p>

In [6]:
SELECT * FROM TD_ColumnSummary (
 ON DEMO_CreditCard.Credit_Card AS InputTable
 USING
 TargetColumns ('[:]')
) AS dt;

,ColumnName,Datatype,NonNullCount,NullCount,BlankCount,ZeroCount,PositiveCount,NegativeCount,NullPercentage,NonNullPercentage
1,TARGET,INTEGER,50000,0,,25175,24825,0,0,100
2,FLAG_MOBIL,INTEGER,50000,0,,1,49999,0,0,100
3,FLAG_OWN_CAR,VARCHAR(10) CHARACTER SET UNICODE,50000,0,0,,,,0,100
4,SK_ID_CURR,BIGINT,50000,0,,0,50000,0,0,100
5,NAME_CONTRACT_TYPE,VARCHAR(15) CHARACTER SET UNICODE,50000,0,0,,,,0,100
6,FLAG_EMP_PHONE,INTEGER,50000,0,,7638,42362,0,0,100
7,NAME_FAMILY_STATUS,VARCHAR(50) CHARACTER SET UNICODE,50000,0,0,,,,0,100
8,AMT_INCOME_TOTAL,FLOAT,50000,0,,0,50000,0,0,100
9,CODE_GENDER,VARCHAR(10) CHARACTER SET UNICODE,50000,0,0,,,,0,100
10,CNT_FAM_MEMBERS,INTEGER,50000,0,,0,50000,0,0,100


<p style = 'font-size:16px;font-family:Arial'>In below sql we are checking the null percentage of columns</p>

In [7]:
SELECT columnname, datatype, nullpercentage FROM TD_ColumnSummary (
 ON DEMO_CreditCard.Credit_Card AS InputTable
 USING
 TargetColumns ('[:]')
) AS dt
where 
nullpercentage > 0
order by 3 desc;

,ColumnName,Datatype,NullPercentage
1,HOUSETYPE_MODE,VARCHAR(50) CHARACTER SET UNICODE,53.08
2,OCCUPATION_TYPE,VARCHAR(50) CHARACTER SET UNICODE,28.582


<p style = 'font-size:16px;font-family:Arial'>As the column HOUSETYPE_MODE has more than 50% of null values we can remove this column from our model calculations.<br>
    Let's check the other varchar columns. 
OCCUPATION_TYPE also has high % of null values.</p>

<p style = 'font-size:16px;font-family:Arial'><b>TD_CATEGORICALSUMMARY </b>function   displays the distinct values and their counts for each specified input table column</p>

In [8]:
Create volatile table cateogrySummaryTable as (
SELECT * FROM TD_CATEGORICALSUMMARY (
ON DEMO_CreditCard.Credit_Card as inputtable
USING
TargetColumns('CODE_GENDER'
,'NAME_CONTRACT_TYPE'
,'NAME_FAMILY_STATUS'
,'FLAG_OWN_CAR'
,'OCCUPATION_TYPE')
) AS dt)With data 
on commit preserve rows;

Success: 0 rows affected

In [9]:
select top 5* from cateogrySummaryTable;

,ColumnName,DistinctValue,DistinctValueCount
1,OCCUPATION_TYPE,Realty agents,128
2,OCCUPATION_TYPE,High skill tech staff,1647
3,CODE_GENDER,F,30943
4,CODE_GENDER,M,19057
5,NAME_FAMILY_STATUS,Married,31175


<p style = 'font-size:16px;font-family:Arial'><b>TD_GETFUTILECOLUMNS </b>function displays the categorical columns which will have no effect on the model i.e if all the values are same or unique or If the count of distinct values in the columns divided by the count of the total number of rows in the input
table is greater than or equal to the threshold value</p>

In [10]:
Select * from TD_getFutileColumns(
ON DEMO_CreditCard.Credit_Card as inputtable partition by any
ON cateogrySummaryTable as categorytable Dimension
USING
CategoricalSummaryColumn('ColumnName') 
ThresholdValue(0.05)
)As dt;

,ColumnName
1,FLAG_OWN_CAR


<p style = 'font-size:16px;font-family:Arial'>Here we can see that FLAG_OWN_CAR will have no effect on the model as all the values in this column are same so we can remove this column from model creation</p>

<p style = 'font-size:16px;font-family:Arial'>Let us check the values in the OCCUPATION_TYPE column to see what we can do for the NULLs in the column 
</p>

In [11]:
SELECT * FROM cateogrySummaryTable where columnname = 'OCCUPATION_TYPE' order by DistinctValueCount desc;

,ColumnName,DistinctValue,DistinctValueCount
1,OCCUPATION_TYPE,,14291
2,OCCUPATION_TYPE,Laborers,10256
3,OCCUPATION_TYPE,Sales staff,5684
4,OCCUPATION_TYPE,Core staff,4060
5,OCCUPATION_TYPE,Drivers,3570
6,OCCUPATION_TYPE,Managers,3057
7,OCCUPATION_TYPE,High skill tech staff,1647
8,OCCUPATION_TYPE,Accountants,1308
9,OCCUPATION_TYPE,Medicine staff,1289
10,OCCUPATION_TYPE,Security staff,1275


<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b> Impute Missing Values  </b> </p>
<p style = 'font-size:16px;font-family:Arial'><b>TD_SimpleImputeFit </b>will output a table with the values that will be used to substitute the missing values<br>
    <b>TD_SimpleImputeTransform</b> will return the input data set with the missing values filled in.
Verify the NULL values have been removed.<br>
    *Note one can also use the Fit table as input to <b>TD_ColumnTransformer</b>

In [12]:
-- fit the SimpleImpute function on categorical columns
SELECT * FROM TD_SimpleImputeFit (
    ON DEMO_CreditCard.Credit_Card as InputTable
    OUT VOLATILE TABLE OutputTable(impute_fit_cat_output)
    USING
    ColsForLiterals ('OCCUPATION_TYPE')
    Literals ('not provided')
) as dt;



Success: 0 rows affected

,TD_INDEX_SIMFIT,TD_TARGETCOLUMN_SIMFIT,TD_NUM_COLVAL_SIMFIT,TD_STR_COLVAL_SIMFIT,TD_ISNUMERIC_SIMFIT
1,13,OCCUPATION_TYPE,,not provided,0


In [13]:
Create volatile table occupationimputetable as (
SELECT * FROM TD_SimpleImputeTransform (
 ON DEMO_CreditCard.Credit_Card as InputTable
 ON impute_fit_cat_output AS FitTable DIMENSION
) AS dt)With data 
on commit preserve rows;

Success: 0 rows affected

<p style = 'font-size:16px;font-family:Arial'>TD_SimpleImputeFit and TD_SimpleImputeTransform function works on integer columns also and we can use them for filling the missing column values based on min/max/mean/median of the values in the column</p>

<p style = 'font-size:16px;font-family:Arial'>In model creation we usually prefer Numerical inputs instead of Characters, now let us check how many distinct values we have for our character columns so that we can encode them in numerals</p>

In [14]:
SELECT columnname,count(distinctvalue) FROM cateogrySummaryTable 
group by 1 order by 2; 

,ColumnName,DistinctValue
1,FLAG_OWN_CAR,1
2,NAME_CONTRACT_TYPE,2
3,CODE_GENDER,2
4,NAME_FAMILY_STATUS,5
5,OCCUPATION_TYPE,18


<p style = 'font-size:16px;font-family:Arial'>We are not using FLAG_OWN_CAR column in model creation, for other columns we can use <b>TD_OneHotEncodingFit</b> and <b>TD_OrdinalEncodingFit</b> and transform functions to convert character categories to numerals, for that we need to check the exact values present in the columns</p>

In [15]:
SELECT columnname,distinctvalue FROM cateogrySummaryTable where columnname in 
('CODE_GENDER', 'NAME_CONTRACT_TYPE')
order by 1; 

,ColumnName,DistinctValue
1,CODE_GENDER,M
2,CODE_GENDER,F
3,NAME_CONTRACT_TYPE,Cash loans
4,NAME_CONTRACT_TYPE,Revolving loans


<p style = 'font-size:28px;font-family:Arial;color:#E37C4D'> <b> 3. Feature Engineering Functions  </b> </p>
<p style = 'font-size:16px;font-family:Arial'><b>TD_OneHotEncodingFit </b>outputs a table of attributes and categorical values to input to <b>TD_OneHotEncodingTransform </b> which encodes them as one-hot numeric vectors.</p>

In [16]:
CREATE VOLATILE TABLE onehotencodingfittable AS (
SELECT * FROM TD_OneHotEncodingFit (
 ON DEMO_CreditCard.Credit_Card AS InputTable
 USING
  TargetColumn ('CODE_GENDER','NAME_CONTRACT_TYPE')
  OtherColumnName ('other')
  IsInputDense ('true')
  CategoryCounts(2,2)
  Approach('Auto')    
 ) AS dt
) WITH DATA
ON COMMIT PRESERVE ROWS;

Success: 0 rows affected

<p style = 'font-size:16px;font-family:Arial'>We can check how the fit table looks like</p>

In [17]:
select * from onehotencodingfittable;

,CODE_GENDER,CODE_GENDER_0,CODE_GENDER_1,CODE_GENDER_other,NAME_CONTRACT_TYPE,NAME_CONTRACT_TYPE_0,NAME_CONTRACT_TYPE_1,NAME_CONTRACT_TYPE_other
1,,,,,,Cash loans,Revolving loans,
2,,F,M,,,,,


<p style = 'font-size:16px;font-family:Arial'>For categorical columns which have many values we can use <b>TD_OrdinalEncoding</b> instead</p>

In [18]:
SELECT * FROM TD_OrdinalEncodingFit (
ON DEMO_CreditCard.Credit_Card AS InputTable
OUT volatile table outputtable (ordinalencodingfittable)
USING
  TargetColumn ('NAME_FAMILY_STATUS','OCCUPATION_TYPE')
  DefaultValue (-1)
) as dt;

Success: 0 rows affected

,TD_ColumnName_ORDFIT,TD_Category_ORDFIT,TD_Value_ORDFIT,TD_Index_ORDFIT,NAME_FAMILY_STATUS,OCCUPATION_TYPE
1,OCCUPATION_TYPE,Accountants,0,1,,
2,NAME_FAMILY_STATUS,Civil marriage,0,0,,
3,OCCUPATION_TYPE,Cleaning staff,1,1,,
4,NAME_FAMILY_STATUS,Married,1,0,,
5,OCCUPATION_TYPE,Cooking staff,2,1,,
6,NAME_FAMILY_STATUS,Separated,2,0,,
7,OCCUPATION_TYPE,Core staff,3,1,,
8,NAME_FAMILY_STATUS,Single / not married,3,0,,
9,OCCUPATION_TYPE,Drivers,4,1,,
10,NAME_FAMILY_STATUS,Widow,4,0,,


<p style = 'font-size:16px;font-family:Arial'><b>TD_BinCodeFit and TD_BinCodeTransform </b>bin-codes the
specified input table columns. Bin-coding is typically used to convert numeric data to categorical data by binning the numeric data into multiple numeric bins (intervals).</p> 

<p style = 'font-size:16px;font-family:Arial'>For variable width bins, we need to provide the bin table to the function. Let's create the table and use that in the TD_BinCodeFit function</p>

In [19]:
create table FitInputTable (ColumnName varchar(20), MinValue integer, MaxValue integer, Label varchar(20));

Success: 0 rows affected

In [20]:
insert into FitInputTable values('age', 0, 18, '1-Children');
insert into FitInputTable values('age', 19, 25, '2-Young Adult');    
insert into FitInputTable values('age', 26, 45, '3-Middle Adult');
insert into FitInputTable values('age', 46, 60, '4-Old Adult');    
insert into FitInputTable values('age', 61 ,120, '5-Senior Citizen');

Success: 1 rows affected

Success: 1 rows affected

Success: 1 rows affected

Success: 1 rows affected

Success: 1 rows affected

In [21]:
create volatile table FitOutputTable as (
SELECT * FROM TD_BincodeFit(
ON DEMO_CreditCard.Credit_Card as InputTable
ON FitInputTable as FitInput Dimension
USING
TargetColumns('age')
MethodType('Variable-Width')
MinValueColumn('MinValue')
MaxValueCOlumn('MaxValue')
LabelColumn('Label')
TargetColNames('ColumnName')
) AS dt
) with data
ON COMMIT PRESERVE ROWS;


Success: 0 rows affected

<p style = 'font-size:16px;font-family:Arial'>The fit table looks like below:</p>

In [22]:
select * from FitOutputTable;

,TD_ColumnName_BINFIT,TD_MinValue_BINFIT,TD_MaxValue_BINFIT,TD_Label_BINFIT,TD_Bins_BINFIT,TD_IndexValue_BINFIT,TD_MaxLenLabel_BINFIT,AGE
1,AGE,46,60,4-Old Adult,5,0,16,
2,AGE,0,18,1-Children,5,0,16,
3,AGE,19,25,2-Young Adult,5,0,16,
4,AGE,61,120,5-Senior Citizen,5,0,16,
5,AGE,26,45,3-Middle Adult,5,0,16,


<p style = 'font-size:16px;font-family:Arial'><b>TD_ScaleFit and TDScaleTransform </b>scales specified input
table columns i.e perform the specific scale methods like standard deviation, mean etc to the input columns </p> 

In [23]:
select * from TD_scaleFit(
on DEMO_CreditCard.Credit_Card  as InputTable
OUT VOLATILE TABLE OutputTable(scaleFitOut)
using
TargetColumns('amt_income_total')
MissValue('Keep')
ScaleMethod('range')
GlobalScale('f')
)as dt;


Success: 0 rows affected

,TD_STATTYPE_SCLFIT,AMT_INCOME_TOTAL
1,min,25650
2,max,1.17e+08
3,sum,8.3701789473125e+09
4,count,50000
5,null,0
6,avg,167403.57894625
7,multiplier,1
8,intercept,0
9,location,25650
10,scale,1.1697435e+08


<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b> TD_ColumnTransformer  </b> </p>
<p style = 'font-size:16px;font-family:Arial'>The TD_ColumnTransformer function transforms the entire dataset in a single operation. You only need
to provide the FIT tables to the function, and the function runs all transformations that you require in a
single operation. Running all the it table transformations together in one-go gives approx. 30% performance improvement over running each transformation sequentially.</p>

<p style = 'font-size:16px;font-family:Arial'>Let us put all the fit tables we have created and transform the dataset</p>

In [24]:
SELECT *
    FROM TD_ColumnTransformer(
 ON DEMO_CreditCard.Credit_Card AS inputtable
 ON  impute_fit_cat_output AS SImpleImputeFitTable DIMENSION   
 ON onehotencodingfittable AS ONehotencodingfittable DIMENSION
 ON ordinalencodingfittable AS OrdinalEncodingFitTable DIMENSION
 ON FitOutputTable AS BincodeFitTable DIMENSION  
 ON scaleFitOut AS ScaleFitTable DIMENSION         
)
AS dt
;


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_FAMILY_STATUS,REGION_POPULATION_RELATIVE,FLAG_MOBIL,FLAG_EMP_PHONE,CNT_FAM_MEMBERS,HOUSETYPE_MODE,OCCUPATION_TYPE,AGE,CODE_GENDER_0,CODE_GENDER_1,CODE_GENDER_other,NAME_CONTRACT_TYPE_0,NAME_CONTRACT_TYPE_1,NAME_CONTRACT_TYPE_other
1,143600,1,Cash loans,F,N,0,0.0005501206033630449,0,0.02280000038444996,1,1,2,,1,3-Middle Adult  ,1,0,0,1,0,0
2,124104,0,Cash loans,F,N,0,0.0007501644591314249,1,0.03579200059175491,1,1,2,,-1,5-Senior Citizen,1,0,0,1,0,0
3,126899,0,Cash loans,F,N,2,0.0013195200486260448,1,0.025164000689983368,1,1,4,block of flats,14,3-Middle Adult  ,1,0,0,1,0,0
4,118537,0,Cash loans,F,N,1,0.001396459993152345,1,0.007019999902695417,1,1,3,,8,4-Old Adult     ,1,0,0,1,0,0
5,101613,0,Cash loans,F,N,0,0.000857880381468245,1,0.03579200059175491,1,0,2,,-1,4-Old Adult     ,1,0,0,1,0,0
6,357714,1,Cash loans,M,N,2,0.0013195200486260448,1,0.03579200059175491,1,0,4,block of flats,-1,3-Middle Adult  ,0,1,0,1,0,0
7,295785,1,Cash loans,F,N,1,0.0006270605478893449,1,0.009657000191509724,1,1,3,block of flats,3,3-Middle Adult  ,1,0,0,1,0,0
8,107403,1,Cash loans,F,N,0,0.000934820325994545,1,0.030755000188946724,1,1,2,,14,3-Middle Adult  ,1,0,0,1,0,0
9,115765,0,Cash loans,F,N,0,0.0006270605478893449,2,0.009657000191509724,1,1,1,,14,4-Old Adult     ,1,0,0,1,0,0
10,119006,0,Cash loans,M,N,2,0.0013195200486260448,2,0.01014699973165989,1,1,3,,-1,3-Middle Adult  ,0,1,0,1,0,0


<p style = 'font-size:16px;font-family:Arial'>We can create a separate intermediate table after performing all the transformations and removing all the columns from the original table which are not needed further.</p>

In [25]:
Create table Transformed_data as(
SELECT SK_ID_CURR,"TARGET",CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_FAMILY_STATUS,REGION_POPULATION_RELATIVE
    ,substr(age,1,1) as "AGE_GROUP",FLAG_MOBIL,FLAG_EMP_PHONE,CNT_FAM_MEMBERS,OCCUPATION_TYPE,CODE_GENDER_1 as Male,CODE_GENDER_0 as Female,
    "NAME_CONTRACT_TYPE_1" as "REVOLVING_LOANS","NAME_CONTRACT_TYPE_0" as "CASH_LOANS"
    FROM TD_ColumnTransformer(
ON DEMO_CreditCard.Credit_Card AS inputtable
ON impute_fit_cat_output AS SImpleImputeFitTable DIMENSION   
ON onehotencodingfittable AS ONehotencodingfittable DIMENSION
ON ordinalencodingfittable AS OrdinalEncodingFitTable DIMENSION  
ON FitOutputTable AS BincodeFitTable DIMENSION  
ON scaleFitOut AS ScaleFitTable DIMENSION   
)
AS dt
) WITH DATA
; 

Success: 0 rows affected

In [26]:
select top 10* from Transformed_data;

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_FAMILY_STATUS,REGION_POPULATION_RELATIVE,AGE_GROUP,FLAG_MOBIL,FLAG_EMP_PHONE,CNT_FAM_MEMBERS,OCCUPATION_TYPE,Male,Female,REVOLVING_LOANS,CASH_LOANS
1,101613,0,0,0.000857880381468245,1,0.03579200059175491,4,1,0,2,-1,0,1,0,1
2,115765,0,0,0.0006270605478893449,2,0.009657000191509724,4,1,1,1,14,0,1,0,1
3,408532,1,1,0.002088919493889045,1,0.02280000038444996,3,1,1,3,8,1,0,0,1
4,127368,0,0,0.002088919493889045,1,0.046220000833272934,4,1,1,2,10,1,0,0,1
5,121801,0,0,0.0005501206033630449,1,0.00886599998921156,4,1,0,2,-1,0,1,0,1
6,130386,0,1,0.004397117829678045,0,0.025164000689983368,3,1,1,3,10,1,0,0,1
7,186525,1,0,0.000742470464678795,1,0.01968899928033352,3,1,1,2,4,1,0,1,0
8,295785,1,1,0.0006270605478893449,1,0.009657000191509724,3,1,1,3,3,0,1,0,1
9,126899,0,2,0.0013195200486260448,1,0.025164000689983368,3,1,1,4,14,0,1,0,1
10,143600,1,0,0.0005501206033630449,0,0.02280000038444996,3,1,1,2,1,0,1,0,1


<p style = 'font-size:16px;font-family:Arial'>Now that we’ve shown you how you can use ClearScape in-database functions for preparing the data, you’ve now got a set of data that is cleansed and processed you could proceed to use this as an input in data science model creation. 
</p>

<p style = 'font-size:28px;font-family:Arial;color:#E37C4D'><b>4.Clean up</b> </p>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Worktables</b> </p>

In [27]:
DROP TABLE Transformed_data;

Success: 29 rows affected

In [28]:
DROP TABLE FitInputTable;

Success: 18 rows affected

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'> <b>Database and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [29]:
call remove_data('DEMO_CreditCard');-- takes about 5 seconds, optional if you want to use the data later

Success: 0 rows affected

Success: 0 rows affected

,Message
1,Removed objects related to DEMO_CreditCard. That ran for 0:00:01.70


<p style = 'font-size:28px;font-family:Arial;color:#E37C4D'> <b> 5. Conclusion </b> </p>
<p style = 'font-size:16px;font-family:Arial'>In this notebook we have seen some of the Teradata Vantage Clearscape's new inDb functions for data cleansing, data exploration and feature engineering. Many of these functions can be applied in one go using the TD_COLUMNTRANSFORM function which gives is approx. 30% faster than serial processing.</p>

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>Reference Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
        <li>Teradata Analytic Function Reference:
        <a href = 'https://docs.teradata.com/r/Teradata-VantageCloud-Lake/Analyzing-Your-Data/Analytics-Database-Analytic-Functions/Analytics-Database-Analytic-Functions-Overview'>
        https://docs.teradata.com/r/Teradata-VantageCloud-Lake/Analyzing-Your-Data/Analytics-Database-Analytic-Functions/Analytics-Database-Analytic-Functions-Overview</a></li>
  
</ul>

<footer style="padding:10px;background:#f9f9f9;border-bottom:3px solid #394851">©2023 Teradata. All Rights Reserved</footer>